In [36]:
import json
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
from random import shuffle
import shutil
import cv2
import yaml

In [13]:
video_list = glob("./train/*")
shuffle(video_list)
train_v = video_list[:120]
valid_v = video_list[120:]
labels = ["UAV"]
labels2id = {
    "UAV": 0,
}

In [34]:
!rm -rf ./yolo_datasets/
!mkdir ./yolo_datasets/
!mkdir ./yolo_datasets/images
!mkdir ./yolo_datasets/labels
!mkdir ./yolo_datasets/images/train
!mkdir ./yolo_datasets/images/valid
!mkdir ./yolo_datasets/labels/train
!mkdir ./yolo_datasets/labels/valid

for video_path in tqdm(train_v):
    video_name = video_path.split('/')[-1]
    json_path = video_path + "/IR_label.json"
    with open(json_path, 'r') as f:
        js = json.load(f)
        image_id = 1
        for is_exist, bbox in zip(js['exist'], js['gt_rect']):
            image_name = "{:0>6}".format(image_id)
            shutil.copyfile(video_path + "/" + image_name + ".jpg", f"./yolo_datasets/images/train/{video_name}_{image_name}.jpg")
            with open(f"./yolo_datasets/labels/train/{video_name}_{image_name}.txt", 'w') as af:
                if ( is_exist == 1 ):
                    img = cv2.imread(video_path + "/" + image_name + ".jpg")
                    height, width, channel = img.shape
                    # x y w h
                    # class x_center, y_center, w, h
                    x_center = (int(bbox[0]) + int(bbox[2]) / 2) / width
                    y_center = (int(bbox[1]) + int(bbox[3]) / 2) / height
                    w = int(bbox[2]) / width
                    h = int(bbox[3]) / height
                    af.write(f"{0} {x_center} {y_center} {w} {h}\n")
            image_id += 1


100%|██████████| 120/120 [04:02<00:00,  2.02s/it]


In [35]:
for video_path in tqdm(valid_v):
    video_name = video_path.split('/')[-1]
    json_path = video_path + "/IR_label.json"
    with open(json_path, 'r') as f:
        js = json.load(f)
        image_id = 1
        for is_exist, bbox in zip(js['exist'], js['gt_rect']):
            image_name = "{:0>6}".format(image_id)
            shutil.copyfile(video_path + "/" + image_name + ".jpg", f"./yolo_datasets/images/valid/{video_name}_{image_name}.jpg")
            with open(f"./yolo_datasets/labels/valid/{video_name}_{image_name}.txt", 'w') as af:
                if ( is_exist == 1 ):
                    img = cv2.imread(video_path + "/" + image_name + ".jpg")
                    height, width, channel = img.shape
                    # x y w h
                    # class x_center, y_center, w, h
                    x_center = (int(bbox[0]) + int(bbox[2]) / 2) / width
                    y_center = (int(bbox[1]) + int(bbox[3]) / 2) / height
                    w = int(bbox[2]) / width
                    h = int(bbox[3]) / height
                    af.write(f"{0} {x_center} {y_center} {w} {h}\n")
            image_id += 1

100%|██████████| 30/30 [00:58<00:00,  1.94s/it]


In [37]:
data = dict(
    train = '/home/frankie/yolo_datasets/images/train',
    val = '/home/frankie/yolo_datasets/images/valid',
    nc    = 1, # number of classes
    names = ["UAV"] # classes
)

with open('./yolo_datasets/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

